In [238]:
#STREAMS2TABLE
#written by Zachary J.J. Roper

#A script for converting raw event codes and timestamps into a trial-item dataset

In [239]:
#This notebook requires the following modules
import os
import numpy
import csv

In [240]:
#READ IN EVENT CODES
#The prepared file, out.csv, is a 2 column x session length (based on total number of trials) consistinting of event 
#codes (EVENTS[]) and their paired timestamps(TIMES[]). The file streams2text.m was used to generate these data 
#from the TDT data dump.

In [241]:
with open('out.csv') as csvfile:
    data = csv.reader(csvfile, delimiter = ',')
    events = []
    times = []
    for row in data:
        event = row[0]
        events.append(event)
        time = row[1]
        times.append(time)
pairedlist = list(zip(events,times))

In [305]:
trial = 0
triallist = [None]*len(pairedlist)
eclist = [None]*len(pairedlist)
for i, data in enumerate(pairedlist):
    if pairedlist[i][0] == '2301':
        trial += 1
    triallist[i] = trial  
    eclist[i] = i
uberlist = list(zip(triallist,eclist,pairedlist))

In [360]:
uberlist

[(0, 0, ('1507', ' 3258.4908')),
 (0, 1, ('1500', ' 3258.6548')),
 (0, 2, ('23', ' 3258.7776')),
 (0, 3, ('2', ' 3258.9414')),
 (0, 4, ('8', ' 3259.0644')),
 (0, 5, ('0', ' 3259.1872')),
 (0, 6, ('2', ' 3260.4160')),
 (0, 7, ('0', ' 3260.5388')),
 (0, 8, ('1666', ' 3278.8889')),
 (1, 9, ('2301', ' 3279.0117')),
 (1, 10, ('0', ' 3279.1348')),
 (1, 11, ('2660', ' 4024.4429')),
 (1, 12, ('0', ' 4024.5657')),
 (1, 13, ('2651', ' 4892.4673')),
 (1, 14, ('0', ' 4892.5900')),
 (1, 15, ('2300', ' 4894.4743')),
 (1, 16, ('0', ' 4894.5971')),
 (1, 17, ('2001', ' 6898.6061')),
 (1, 18, ('776', ' 6898.7699')),
 (1, 19, ('1667', ' 6898.8929')),
 (1, 20, ('2998', ' 6899.0567')),
 (1, 21, ('3999', ' 6899.1795')),
 (1, 22, ('0', ' 6899.3433')),
 (1, 23, ('4001', ' 6900.4903')),
 (1, 24, ('4052', ' 6900.6131')),
 (1, 25, ('4060', ' 6900.7769')),
 (1, 26, ('4108', ' 6900.8999')),
 (1, 27, ('4152', ' 6901.0227')),
 (1, 28, ('0', ' 6901.1457')),
 (1, 29, ('4201', ' 6902.4563')),
 (1, 30, ('4258', ' 6902.5

In [353]:
'1507' in uberlist[0][2]

True

In [355]:
RT = [None]*len(pairedlist)
for i, trial in enumerate(uberlist):
    print(trial)
    if '2651' in uberlist[i][2]:
        RTbeg = float(uberlist[i][2][1])
        focusTrial = uberlist[i][0]
        for p, case in enumerate(uberlist):
            if '2810' in uberlist[p][2] and uberlist[i][2] == focusTrial:
                RTend = float(uberlist[p][2][1])
    RT[i] = RTend - RTbeg

0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8


75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
75
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
76
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
77
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
78
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
81
8

132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
133
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
134
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
135
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136


168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
169
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
171
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172
172


208
208
208
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212


...

673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
673
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
674
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
675
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
676
677
677
677
677
677
677
677
677
677
677
677
677
677
677
677
677
677
677
677
677
677
677


720
720
720
720
720
720
720
720
720
720
720
720
720
720
720
720
720
720
720
720
720
720
720
720
720
720
720
720
720
720
720
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
721
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
722
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
723
724
724
724
724
724
724
724
724
724
724
724
724
724
724
724
724
724
724
724
724
724
724
724
724
724


769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
769
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
770
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
771
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
772
773
773
773
773
773
773
773
773
773
773
773
773
773


813
813
813
813
813
813
813
813
813
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
814
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
815
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
816
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817
817


850
850
850
850
850
850
850
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
851
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
852
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
853
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854
854


In [351]:
RT

[178.01220000023022,
 178.01220000023022,
 178.01220000023022,
 178.01220000023022,
 178.01220000023022,
 178.01220000023022,
 178.01220000023022,
 178.01220000023022,
 178.01220000023022,
 178.01220000023022,
 178.01220000023022,
 178.01220000023022,
 178.01220000023022,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,
 2272191.324,

In [242]:
#The following provides indexed lists for each of the individual event codes of interest 

In [243]:
TargetOnset = []
FixOnset = []
Saccade = []
for i, x in enumerate(events):
    print(i,x)
    if x == '2301':
        FixOnset.append(i)
    if x == '2651':
        TargetOnset.append(i)
    if x == '2810':
        Saccade.append(i)

0 1507
1 1500
2 23
3 2
4 8
5 0
6 2
7 0
8 1666
9 2301
10 0
11 2660
12 0
13 2651
14 0
15 2300
16 0
17 2001
18 776
19 1667
20 2998
21 3999
22 0
23 4001
24 4052
25 4060
26 4108
27 4152
28 0
29 4201
30 4258
31 4300
32 4350
33 4400
34 0
35 5135
36 5680
37 3800
38 4650
39 4663
40 0
41 4673
42 4700
43 4800
44 4901
45 6000
46 0
47 3003
48 3200
49 5000
50 11000
51 3100
52 0
53 3600
54 3100
55 3030
56 4500
57 4000
58 0
59 4500
60 4000
61 4690
62 4790
63 2999
64 0
65 1666
66 2301
67 0
68 2660
69 0
70 2651
71 0
72 2300
73 0
74 2810
75 0
76 888
77 0
78 2600
79 0
80 2001
81 777
82 0
83 2727
84 1667
85 2998
86 3999
87 4001
88 0
89 4052
90 4060
91 4108
92 4152
93 4201
94 0
95 4258
96 4300
97 4350
98 4400
99 5180
100 0
101 5725
102 3800
103 4650
104 4663
105 4673
106 0
107 4700
108 4800
109 4901
110 6000
111 3007
112 0
113 3200
114 5000
115 11000
116 3100
117 3600
118 0
119 3100
120 3030
121 4500
122 4000
123 4500
124 0
125 4000
126 4690
127 4790
128 2999
129 0
130 1666
131 2301
132 0
133 2660
134 0
135

2252 0
2253 888
2254 0
2255 2600
2256 0
2257 2001
2258 777
2259 0
2260 2727
2261 1667
2262 2998
2263 3999
2264 4001
2265 0
2266 4052
2267 4060
2268 4108
2269 4152
2270 4201
2271 0
2272 4258
2273 4300
2274 4350
2275 4400
2276 5135
2277 0
2278 5680
2279 3800
2280 4650
2281 4663
2282 4673
2283 0
2284 4700
2285 4800
2286 4901
2287 6000
2288 3007
2289 0
2290 3200
2291 5000
2292 11000
2293 3100
2294 3600
2295 0
2296 3100
2297 3030
2298 4500
2299 4000
2300 4500
2301 0
2302 4000
2303 4690
2304 4790
2305 2999
2306 0
2307 1666
2308 2301
2309 0
2310 2660
2311 0
2312 2651
2313 0
2314 2300
2315 0
2316 2810
2317 0
2318 888
2319 0
2320 2600
2321 0
2322 2001
2323 777
2324 0
2325 2727
2326 1667
2327 2998
2328 3999
2329 4001
2330 0
2331 4052
2332 4060
2333 4108
2334 4152
2335 4201
2336 0
2337 4258
2338 4300
2339 4350
2340 4400
2341 5315
2342 0
2343 5500
2344 3800
2345 4650
2346 4663
2347 4673
2348 0
2349 4700
2350 4800
2351 4901
2352 6000
2353 3007
2354 0
2355 3200
2356 5000
2357 11000
2358 3100
2359 36

3603 4000
3604 4500
3605 0
3606 4000
3607 4690
3608 4790
3609 2999
3610 0
3611 1666
3612 2301
3613 2660
3614 0
3615 2620
3616 1667
3617 2998
3618 3999
3619 4001
3620 0
3621 4052
3622 4060
3623 4108
3624 4152
3625 4201
3626 0
3627 4258
3628 4300
3629 4350
3630 4400
3631 5225
3632 0
3633 5770
3634 3800
3635 4650
3636 4663
3637 4673
3638 0
3639 4700
3640 4800
3641 4901
3642 6000
3643 3002
3644 0
3645 3200
3646 5000
3647 11000
3648 3100
3649 3600
3650 0
3651 3100
3652 3030
3653 4500
3654 4000
3655 4500
3656 0
3657 4000
3658 4690
3659 4790
3660 2999
3661 0
3662 1666
3663 2301
3664 0
3665 2660
3666 0
3667 2620
3668 1667
3669 2998
3670 3999
3671 4001
3672 0
3673 4052
3674 4060
3675 4108
3676 4152
3677 4201
3678 0
3679 4258
3680 4300
3681 4350
3682 4400
3683 5270
3684 0
3685 5815
3686 3800
3687 4650
3688 4663
3689 4673
3690 0
3691 4700
3692 4800
3693 4901
3694 6000
3695 3002
3696 0
3697 3200
3698 5000
3699 11000
3700 3100
3701 3600
3702 0
3703 3100
3704 3030
3705 4500
3706 4000
3707 4500
3708 

5105 3600
5106 0
5107 3100
5108 3030
5109 4500
5110 4000
5111 4500
5112 0
5113 4000
5114 4690
5115 4790
5116 2999
5117 0
5118 1666
5119 2301
5120 0
5121 2660
5122 0
5123 2620
5124 1667
5125 2998
5126 3999
5127 4001
5128 0
5129 4052
5130 4060
5131 4108
5132 4152
5133 4201
5134 0
5135 4258
5136 4300
5137 4350
5138 4400
5139 5180
5140 0
5141 5725
5142 3800
5143 4650
5144 4663
5145 4673
5146 0
5147 4700
5148 4800
5149 4901
5150 6000
5151 3002
5152 0
5153 3200
5154 5000
5155 11000
5156 3100
5157 3600
5158 0
5159 3100
5160 3030
5161 4500
5162 4000
5163 4500
5164 0
5165 4000
5166 4690
5167 4790
5168 2999
5169 0
5170 1666
5171 2301
5172 0
5173 2620
5174 1667
5175 2998
5176 3999
5177 4001
5178 0
5179 4052
5180 4060
5181 4108
5182 4152
5183 4201
5184 0
5185 4258
5186 4300
5187 4350
5188 4400
5189 5180
5190 0
5191 5725
5192 3800
5193 4650
5194 4663
5195 4673
5196 0
5197 4700
5198 4800
5199 4901
5200 6000
5201 3001
5202 0
5203 3200
5204 5000
5205 11000
5206 3100
5207 3600
5208 0
5209 3100
5210 303

6341 887
6342 0
6343 2001
6344 776
6345 1667
6346 2998
6347 3999
6348 0
6349 4001
6350 4052
6351 4060
6352 4108
6353 4152
6354 0
6355 4201
6356 4258
6357 4300
6358 4350
6359 4400
6360 0
6361 5225
6362 5770
6363 3800
6364 4650
6365 4663
6366 0
6367 4673
6368 4700
6369 4800
6370 4901
6371 6000
6372 0
6373 3005
6374 3200
6375 5000
6376 11000
6377 3100
6378 0
6379 3600
6380 3100
6381 3030
6382 4500
6383 4000
6384 0
6385 4500
6386 4000
6387 4690
6388 4790
6389 2999
6390 0
6391 1666
6392 2301
6393 0
6394 2660
6395 0
6396 2620
6397 1667
6398 2998
6399 3999
6400 4001
6401 0
6402 4052
6403 4060
6404 4108
6405 4152
6406 4201
6407 0
6408 4258
6409 4300
6410 4350
6411 4400
6412 5045
6413 0
6414 5590
6415 3800
6416 4650
6417 4663
6418 4673
6419 0
6420 4700
6421 4800
6422 4901
6423 6000
6424 3002
6425 0
6426 3200
6427 5000
6428 11000
6429 3100
6430 3600
6431 0
6432 3100
6433 3030
6434 4500
6435 4000
6436 4500
6437 0
6438 4000
6439 4690
6440 4790
6441 2999
6442 0
6443 1666
6444 2301
6445 0
6446 2660


...

46346 3030
46347 4500
46348 4000
46349 4500
46350 0
46351 4000
46352 4690
46353 4790
46354 2999
46355 0
46356 1666
46357 2301
46358 0
46359 2660
46360 0
46361 2651
46362 0
46363 2300
46364 0
46365 2810
46366 0
46367 888
46368 0
46369 2600
46370 0
46371 2001
46372 777
46373 0
46374 2727
46375 1667
46376 2998
46377 3999
46378 4001
46379 0
46380 4052
46381 4060
46382 4108
46383 4152
46384 4201
46385 0
46386 4258
46387 4300
46388 4350
46389 4400
46390 5045
46391 0
46392 5590
46393 3800
46394 4650
46395 4663
46396 4673
46397 0
46398 4700
46399 4800
46400 4908
46401 6000
46402 3007
46403 0
46404 3200
46405 5000
46406 11000
46407 3100
46408 3600
46409 0
46410 3100
46411 3030
46412 4500
46413 4000
46414 4500
46415 0
46416 4000
46417 4690
46418 4790
46419 2999
46420 0
46421 1666
46422 2301
46423 0
46424 2660
46425 0
46426 2651
46427 0
46428 2300
46429 0
46430 2810
46431 0
46432 888
46433 0
46434 2600
46435 0
46436 2001
46437 777
46438 0
46439 2727
46440 1667
46441 2998
46442 3999
46443 4001
464

48073 2660
48074 0
48075 2651
48076 0
48077 2300
48078 0
48079 2810
48080 0
48081 888
48082 0
48083 2600
48084 0
48085 2001
48086 777
48087 0
48088 2727
48089 1667
48090 2998
48091 3999
48092 4001
48093 0
48094 4052
48095 4060
48096 4108
48097 4152
48098 4201
48099 0
48100 4258
48101 4300
48102 4350
48103 4400
48104 5090
48105 0
48106 5635
48107 3800
48108 4650
48109 4663
48110 4673
48111 0
48112 4700
48113 4800
48114 4908
48115 6000
48116 3007
48117 0
48118 3200
48119 5000
48120 11000
48121 3100
48122 3600
48123 0
48124 3100
48125 3030
48126 4500
48127 4000
48128 4500
48129 0
48130 4000
48131 4690
48132 4790
48133 2999
48134 0
48135 1666
48136 2301
48137 0
48138 2660
48139 0
48140 2651
48141 0
48142 2300
48143 0
48144 2810
48145 0
48146 888
48147 0
48148 2001
48149 776
48150 1667
48151 2998
48152 3999
48153 0
48154 4001
48155 4052
48156 4060
48157 4108
48158 4152
48159 0
48160 4201
48161 4258
48162 4300
48163 4350
48164 4400
48165 0
48166 5135
48167 5680
48168 3800
48169 4650
48170 46

49727 4790
49728 2999
49729 0
49730 1666
49731 2301
49732 0
49733 2660
49734 0
49735 2651
49736 0
49737 2300
49738 0
49739 2810
49740 0
49741 888
49742 0
49743 2600
49744 0
49745 2001
49746 777
49747 0
49748 2727
49749 1667
49750 2998
49751 3999
49752 4001
49753 0
49754 4052
49755 4060
49756 4108
49757 4152
49758 4201
49759 0
49760 4258
49761 4300
49762 4350
49763 4400
49764 5225
49765 0
49766 5770
49767 3800
49768 4650
49769 4663
49770 4673
49771 0
49772 4700
49773 4800
49774 4908
49775 6000
49776 3007
49777 0
49778 3200
49779 5000
49780 11000
49781 3100
49782 3600
49783 0
49784 3100
49785 3030
49786 4500
49787 4000
49788 4500
49789 0
49790 4000
49791 4690
49792 4790
49793 2999
49794 0
49795 1666
49796 2301
49797 0
49798 2660
49799 0
49800 2651
49801 0
49802 2300
49803 0
49804 2810
49805 0
49806 888
49807 0
49808 2600
49809 0
49810 2001
49811 777
49812 0
49813 2727
49814 1667
49815 2998
49816 3999
49817 4001
49818 0
49819 4052
49820 4060
49821 4108
49822 4152
49823 4201
49824 0
49825 

51304 4673
51305 4700
51306 4800
51307 4909
51308 6000
51309 0
51310 3005
51311 3200
51312 5000
51313 11000
51314 3100
51315 0
51316 3600
51317 3100
51318 3030
51319 4500
51320 4000
51321 0
51322 4500
51323 4000
51324 4690
51325 4790
51326 2999
51327 0
51328 1666
51329 2301
51330 0
51331 2660
51332 0
51333 2620
51334 1667
51335 2998
51336 3999
51337 4001
51338 0
51339 4052
51340 4060
51341 4108
51342 4152
51343 4201
51344 0
51345 4258
51346 4300
51347 4350
51348 4400
51349 5270
51350 0
51351 5815
51352 3800
51353 4650
51354 4663
51355 4673
51356 0
51357 4700
51358 4800
51359 4909
51360 6000
51361 3002
51362 0
51363 3200
51364 5000
51365 11000
51366 3100
51367 3600
51368 0
51369 3100
51370 3030
51371 4500
51372 4000
51373 4500
51374 0
51375 4000
51376 4690
51377 4790
51378 2999
51379 0
51380 1666
51381 2301
51382 0
51383 2660
51384 0
51385 2651
51386 0
51387 2300
51388 0
51389 2810
51390 0
51391 888
51392 0
51393 2600
51394 0
51395 2001
51396 777
51397 0
51398 2727
51399 1667
51400 2998

52771 5045
52772 0
52773 5590
52774 3800
52775 4650
52776 4663
52777 4673
52778 0
52779 4700
52780 4800
52781 4909
52782 6000
52783 3007
52784 0
52785 3200
52786 5000
52787 11000
52788 3100
52789 3600
52790 0
52791 3100
52792 3030
52793 4500
52794 4000
52795 4500
52796 0
52797 4000
52798 4690
52799 4790
52800 2999
52801 0
52802 1666
52803 2301
52804 0
52805 2660
52806 0
52807 2651
52808 0
52809 2300
52810 0
52811 2810
52812 0
52813 888
52814 0
52815 2600
52816 0
52817 2001
52818 777
52819 0
52820 2727
52821 1667
52822 2998
52823 3999
52824 4001
52825 0
52826 4052
52827 4060
52828 4108
52829 4152
52830 4201
52831 0
52832 4258
52833 4300
52834 4350
52835 4400
52836 5315
52837 0
52838 5500
52839 3800
52840 4650
52841 4663
52842 4673
52843 0
52844 4700
52845 4800
52846 4909
52847 6000
52848 3007
52849 0
52850 3200
52851 5000
52852 11000
52853 3100
52854 3600
52855 0
52856 3100
52857 3030
52858 4500
52859 4000
52860 4500
52861 0
52862 4000
52863 4690
52864 4790
52865 2999
52866 0
52867 1666

In [256]:
float(times[74])-float(times[70])

197.95960000000014

In [245]:
Saccade

[74,
 139,
 204,
 268,
 333,
 398,
 514,
 575,
 640,
 705,
 766,
 876,
 1045,
 1110,
 1175,
 1240,
 1305,
 1366,
 1431,
 1496,
 1561,
 1622,
 1683,
 1800,
 1865,
 1930,
 1995,
 2060,
 2125,
 2190,
 2251,
 2316,
 2689,
 2750,
 2866,
 2983,
 3151,
 3930,
 3995,
 4060,
 4125,
 4190,
 4307,
 4372,
 5001,
 5062,
 5229,
 5346,
 5912,
 5977,
 6042,
 6107,
 6172,
 6339,
 6452,
 6976,
 7291,
 7356,
 7421,
 7486,
 7603,
 7771,
 7836,
 7901,
 7966,
 8031,
 8096,
 8209,
 8836,
 8901,
 8965,
 9082,
 9147,
 9212,
 9277,
 9342,
 9407,
 9626,
 9687,
 9748,
 9864,
 9925,
 9990,
 10106,
 10223,
 10340,
 10405,
 10470,
 10535,
 10600,
 10715,
 10882,
 11100,
 11161,
 11226,
 11342,
 11458,
 11574,
 11639,
 11704,
 11765,
 11830,
 11947,
 12012,
 12077,
 12142,
 12207,
 12320,
 12385,
 12450,
 12515,
 12580,
 13356,
 13523,
 13588,
 13653,
 13718,
 13779,
 13844,
 13909,
 14027,
 14088,
 14149,
 14214,
 14279,
 14395,
 14456,
 14521,
 14586,
 14651,
 14819,
 14884,
 14949,
 15061,
 15126,
 15191,
 15256,


In [246]:
for i, x in enumerate(Saccade):
    print(Saccade[i],TargetOnset[i])

74 13
139 70
204 135
268 200
333 264
398 329
514 394
575 510
640 571
705 636
766 701
876 762
1045 872
1110 1041
1175 1106
1240 1171
1305 1236
1366 1301
1431 1362
1496 1427
1561 1492
1622 1557
1683 1618
1800 1679
1865 1796
1930 1861
1995 1926
2060 1991
2125 2056
2190 2121
2251 2186
2316 2247
2689 2312
2750 2685
2866 2746
2983 2862
3151 2979
3930 3147
3995 3926
4060 3991
4125 4056
4190 4121
4307 4186
4372 4303
5001 4368
5062 4997
5229 5058
5346 5225
5912 5342
5977 5908
6042 5973
6107 6038
6172 6103
6339 6168
6452 6335
6976 6448
7291 6972
7356 7287
7421 7352
7486 7417
7603 7482
7771 7599
7836 7767
7901 7832
7966 7897
8031 7962
8096 8027
8209 8092
8836 8205
8901 8832
8965 8897
9082 8961
9147 9078
9212 9143
9277 9208
9342 9273
9407 9338
9626 9403
9687 9622
9748 9683
9864 9744
9925 9860
9990 9921
10106 9986
10223 10102
10340 10219
10405 10336
10470 10401
10535 10466
10600 10531
10715 10596
10882 10711
11100 10878
11161 11096
11226 11157
11342 11222
11458 11338
11574 11454
11639 11570
11704 1

In [247]:
TOtimes = []
for items in TargetOnset:
    TOtimes.append(times[items])

In [248]:
Trials = []
TimeTrials = []
for i, x in enumerate(FixOnset):
    if i < len(FixOnset)-1:
        a = FixOnset[i]
        b = FixOnset[i+1]-1
        Trials.append(events[a:b])
        TimeTrials.append(times[a:b])
    else:
        a = FixOnset[i]
        Trials.append(events[a:])
        TimeTrials.append(times[a:])

In [249]:
print(TimeTrials[0])

[' 3279.0117', ' 3279.1348', ' 4024.4429', ' 4024.5657', ' 4892.4673', ' 4892.5900', ' 4894.4743', ' 4894.5971', ' 6898.6061', ' 6898.7699', ' 6898.8929', ' 6899.0567', ' 6899.1795', ' 6899.3433', ' 6900.4903', ' 6900.6131', ' 6900.7769', ' 6900.8999', ' 6901.0227', ' 6901.1457', ' 6902.4563', ' 6902.5793', ' 6902.7431', ' 6902.8659', ' 6903.0297', ' 6903.1527', ' 6904.4633', ' 6904.6271', ' 6904.7501', ' 6904.9139', ' 6905.0367', ' 6905.1597', ' 6906.4703', ' 6906.5933', ' 6906.7571', ' 6906.9209', ' 6907.0439', ' 6907.1667', ' 6908.4775', ' 6908.6003', ' 6908.7641', ' 6908.8871', ' 6909.0509', ' 6909.1737', ' 6910.4435', ' 6910.6073', ' 6910.7301', ' 6910.8942', ' 6911.0169', ' 6911.1397', ' 6912.4916', ' 6912.6554', ' 6912.7781', ' 6912.9422', ' 6913.0650', ' 6913.1877']


In [250]:
path = os.getcwd()
csvfile = (path+"/table.csv")
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(Trials)

In [251]:
accuracy = [None]*len(Trials)
abort = [None]*len(Trials)
error = [None]*len(Trials)
catch = [None]*len(Trials)
for i, trial in enumerate(Trials):
    if '2600' in trial:
        accuracy[i] = 1 #Correct Saccade (2600)
    else:
        accuracy[i] = 0  
    if '2651' in trial: #fixation was maintained sufficiently long enough for target onset (2651)
        abort[i] = 0
    else:
        abort[i] = 1
    if '887' in trial:
        error[i] = 1
    else:
        error[i] = 0
    if '2757' in trial:
        catch[i] = 1
    else:
        catch[i] = 0

In [267]:
new_dict = {}
for i, trial in enumerate(Trials):
    new_dict[i+1] = dict(zip(Trials[i],TimeTrials[i]))

In [382]:
RT = [None]*len(Trials)
focusTrial = 0
for i, trial in enumerate(Trials):
    if '2651' in trial and '888' in trial and '2600' in trial:
        focusTrial = i + 1
        for p, case in enumerate(uberlist):
            if uberlist[p][0] == focusTrial:
                if '2810' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTend = float(uberlist[p][2][1])
                if '2651' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTbeg = float(uberlist[p][2][1])
            RT[i] = RTend - RTbeg

In [380]:
RT

[None,
 197.95960000000014,
 213.97519999999895,
 204.02170000000115,
 224.01019999999698,
 223.96919999999955,
 203.98070000000007,
 None,
 176.0050999999985,
 183.99240000000282,
 248.01280000000406,
 219.95520000000397,
 379.9861000000019,
 None,
 178.01209999999992,
 None,
 None,
 197.95989999999438,
 165.97000000000116,
 170.02489999998943,
 169.9841000000015,
 217.9481999999989,
 289.9968999999983,
 227.9835000000021,
 188.0063999999984,
 None,
 261.9801000000007,
 195.99349999999686,
 None,
 222.00320000000647,
 214.01600000000326,
 189.9725999999937,
 229.94950000000244,
 180.01909999999043,
 231.99749999999767,
 None,
 287.9896000000008,
 171.99120000001858,
 None,
 None,
 None,
 None,
 None,
 None,
 1763.983500000002,
 213.97500000000582,
 None,
 253.95209999999497,
 None,
 165.9701000000059,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 243.9988000000012,
 210.00190000000293,
 243.9988000000012,
 225.9

In [253]:
checksum = sum(error)+sum(abort)+sum(catch)

In [254]:
checksum

325

In [383]:
for i,x in enumerate(abort):
    print(i+1,abort[i],accuracy[i],error[i],RT[i])

1 0 0 0 None
2 0 1 0 197.95960000000014
3 0 1 0 213.97519999999895
4 0 1 0 204.02170000000115
5 0 1 0 224.01019999999698
6 0 1 0 223.96919999999955
7 0 1 0 203.98070000000007
8 1 0 0 None
9 0 0 0 None
10 0 1 0 183.99240000000282
11 0 1 0 248.01280000000406
12 0 0 0 None
13 0 0 0 None
14 1 0 0 None
15 0 1 0 178.01209999999992
16 1 0 0 None
17 1 0 0 None
18 0 1 0 197.95989999999438
19 0 1 0 165.97000000000116
20 0 1 0 170.02489999998943
21 0 1 0 169.9841000000015
22 0 0 0 None
23 0 1 0 289.9968999999983
24 0 1 0 227.9835000000021
25 0 1 0 188.0063999999984
26 0 0 1 None
27 0 0 0 None
28 0 1 0 195.99349999999686
29 1 0 0 None
30 0 1 0 222.00320000000647
31 0 1 0 214.01600000000326
32 0 1 0 189.9725999999937
33 0 1 0 229.94950000000244
34 0 1 0 180.01909999999043
35 0 1 0 231.99749999999767
36 0 0 1 None
37 0 1 0 287.9896000000008
38 0 1 0 171.99120000001858
39 1 0 0 None
40 1 0 0 None
41 1 0 0 None
42 1 0 0 None
43 1 0 0 None
44 1 0 0 None
45 0 0 0 None
46 0 1 0 213.97500000000582
47 1 0 

402 0 1 0 181.98540000012144
403 1 0 0 None
404 0 1 0 185.91760000004433
405 0 1 0 197.9597000000067
406 0 1 0 219.99600000004284
407 0 1 0 183.99240000010468
408 1 0 0 None
409 1 0 0 None
410 0 1 0 202.0146999999415
411 1 0 0 None
412 0 1 0 203.98070000018924
413 1 0 0 None
414 0 1 0 149.99560000002384
415 0 1 0 191.9794000000693
416 0 1 0 164.00379999983124
417 0 1 0 205.98789999983273
418 0 1 0 188.00640000007115
419 1 0 0 None
420 0 1 0 200.00769999995828
421 1 0 0 None
422 0 1 0 203.9806999999564
423 0 1 0 199.96669999998994
424 0 0 0 None
425 0 1 0 212.00890000001527
426 0 1 0 121.97879999992438
427 0 1 0 231.99739999999292
428 0 1 0 173.99789999984205
429 0 1 0 179.97830000007525
430 0 1 0 189.97240000008605
431 0 0 0 None
432 1 0 0 None
433 0 1 0 200.00759999989532
434 0 1 0 207.95389999984764
435 0 0 1 None
436 0 1 0 205.98790000006557
437 0 0 0 None
438 1 0 0 None
439 0 1 0 227.98320000013337
440 1 0 0 None
441 0 1 0 179.89609999977984
442 0 1 0 164.0038999998942
443 0 1 0 20